# Getting Spotify API Token

In [36]:
import requests
import base64
import datetime

from urllib.parse import urlencode

In [94]:
client_id = 'temp'
client_secret = 'temp'

# Spotify API Class

In [97]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    expired = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
        
    def get_client_credentials(self):
        '''
        Return a base-64 encoded string
        '''
        
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret.")
        
        
        client_creds = f"{self.client_id}:{self.client_secret}"
            
        client_creds_base64 = base64.b64encode(client_creds.encode())
        
        return client_creds_base64.decode()
        
    def get_token_headers(self):
        
        client_creds_base64 = self.get_client_credentials()
        
        token_headers = {
    "Authorization":f"Basic {client_creds_base64}", #base 64 encoded string
}
        
        return token_headers
    
    def get_token_data(self):
        
        token_data = {
    "grant_type":"client_credentials"
}
        
        return token_data
    
    def perform_auth(self):
        
        r = requests.post(self.token_url, data = self.get_token_data(), headers = self.get_token_headers())
        token_response_data = r.json()
        
        if r.status_code not in range(200, 299):
            return False
        
        now = datetime.datetime.now()
        self.access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token_expires = expires
        self.expired = expires < now
        return True
    
    def get_token(self):
        auth_done = self.perform_auth()
        if not auth_done:
            raise Exception("Authentication Failed")
        token = self.access_token 
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_token()
        return token 
    
    def get_resource_header(self):
        headers = {
            "Authorization": f"Bearer {self.get_token()}"
        }
        return headers

    def get_resource(self, lookup_id, resource_type = 'albums', version = 'v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type = 'albums', version = 'v1')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type = 'artists', version = 'v1')
    
    
    def base_search(self, q):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"

        lookup_url = f"{endpoint}?{q}"
        r = requests.get(lookup_url, headers = headers)
        
        if r.status_code not in range(200, 299):
            return {}
        
        return r.json()
    
    def search(self, query = None, operator = None, operator_query = None, search_type = 'artist'):
        
        if query == None:
            raise Exception("A query is required")

        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
            
        query_params = urlencode({"q": str(query), "type": str(search_type.lower())})
        
        return self.base_search(query_params)